# Import Block

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from transformers import AutoTokenizer, AutoModelWithLMHead, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier #not using K nearest
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, roc_curve, auc, classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
import spacy
import accelerate
import contractions
import math
import re
import numpy
import heapq
import warnings
import gensim
from gensim.models import Word2Vec
lemmatizer = WordNetLemmatizer()
tfidf = TfidfVectorizer(min_df=5, max_features=200)
CountVectorizer = CountVectorizer()
tokenizer = AutoTokenizer.from_pretrained('gpt2') 
model = AutoModelWithLMHead.from_pretrained('gpt2')
warnings.filterwarnings(action = 'ignore')
stopwords = set(stopwords.words('english'))
stopwords.remove('no')
stopwords.remove('not')
stopwords.remove('nor')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avamc_srsebwe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\avamc_srsebwe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avamc_srsebwe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
d:\anaconda\envs\model_stuf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'statsmodels'

# Read Dataset

In [80]:
dataset = pd.read_csv('./data/quotes.csv')

# train = train.truncate(before=750000, after=850000)
# train = train.sample(frac=.1, random_state = 8)

#drop na cells
dataset = dataset.dropna()
dataset.isna().sum()

quote       0
author      0
category    0
dtype: int64

# Text Preprocessing

Methods Used:
1. change to lower
2. expand contractions
3. remove special chars, punctuation, and digital numbers
4. remove stop words except not, no, and nor (done in lemm+token section)
5. Limit sentence length to 1024

In [81]:
def cleanTheData(data):
    #data is one string from df
    if(len(data)) > 100:
        return
    #expand contractions and go to lowercase
    data = contractions.fix(data.lower()) 
    data = re.sub('[^a-z]', ' ', data)
    #data = data[0:100]
    return data

#quick sample test run
'''
df2= test = pd.read_csv('test.csv')
df2["A"] = df2['Text'].apply(cleanTheData)
df2.head()
'''
        
dataset['quote'] = dataset['quote'].apply(cleanTheData)

dataset = dataset.dropna()
dataset['quote'].to_csv("./data/clean.csv")

Convert csv to text file and split

In [110]:
clean = pd.read_csv('./data/clean.csv')

In [111]:
test = clean[0:math.floor(.5*len(clean))]
train = clean[math.ceil(.5*len(clean)):len(clean)]

In [112]:
len(test)

96522

In [117]:
train_path = './data/train_clean.txt'
test_path = './data/test_clean.txt'

bos = '<|bos|>'
eos = '<|eos|>'
pad = '<|pad|>'
special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad}
tokenizer.add_special_tokens(special_tokens_dict)

with open(train_path, "w", encoding='utf-8') as f:
    for i in train['quote']:
        f.write (tokenizer.special_tokens_map['bos_token']+i.rstrip()+ "\n" + tokenizer.special_tokens_map['eos_token'])
        
with open(test_path, "w", encoding='utf-8') as f:
    for i in test['quote']:
        f.write (tokenizer.special_tokens_map['bos_token']+ i.rstrip() + "\n" + tokenizer.special_tokens_map['eos_token'])

In [118]:
i = 0
with open(test_path, "r") as read:
    for line in read.readlines():
        i = i+1
        
print(i)

96523


# Train the Model

In [103]:
tokenizer = AutoTokenizer.from_pretrained('gpt2') 
model = AutoModelWithLMHead.from_pretrained('gpt2')

In [119]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer, 
    mlm=False)

train_data = TextDataset(
    tokenizer=tokenizer, 
    file_path=train_path, 
    block_size=1024)

test_data = TextDataset(
    tokenizer=tokenizer, 
    file_path=test_path, 
    block_size=1024)

In [121]:
print(len(test_data))

2223


training

In [123]:
model_path = './model-all'

training_args = TrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_gpu_train_batch_size=32, # batch size for training
    per_gpu_eval_batch_size=64,  # batch size for evaluation
    logging_steps = 500, # Number of update steps between two evaluations.
    save_steps=500, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=test_data
)

trainer.train() 
trainer.save_model(model_path) 
tokenizer.save_pretrained(model_path)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


KeyboardInterrupt: 

# Small sized example for testing

In [100]:
train_path = './data/train_clean.txt'
test_path = './data/test_clean.txt'

s_train_path = './data/small_train_clean.txt'
s_test_path = './data/small_test_clean.txt'

i = 50000
temp = ""
with open(train_path, "r", encoding='utf-8') as read:
    for line in read.readlines():
        if (i == 0):
            continue 
        temp += line
        i = i-1
with open(s_train_path, "w", encoding='utf-8') as f:
    f.write(temp)

i = 50000
temp = "" 
with open(test_path, "r", encoding='utf-8') as read:
    for line in read.readlines():
        if (i == 0):
            continue 
        temp += line
        i = i-1
with open(s_test_path, "w", encoding='utf-8') as f:
    f.write(temp)
    
    
data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer, 
    mlm=False)
    
s_train_data = TextDataset(
    tokenizer=tokenizer, 
    file_path=s_train_path, 
    block_size=256)

s_test_data = TextDataset(
    tokenizer=tokenizer, 
    file_path=s_test_path, 
    block_size=256)

In [101]:
len(s_test_data)

5876

In [102]:
model_path = './model-all'
training_args = TrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_gpu_train_batch_size=32, # batch size for training
    per_gpu_eval_batch_size=64,  # batch size for evaluation
    logging_steps = 500, # Number of update steps between two evaluations.
    save_steps=500, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=s_train_data,
    eval_dataset=s_test_data
)

trainer.train() 
trainer.save_model(model_path) 
tokenizer.save_pretrained(model_path)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


  0%|          | 0/1900 [01:41<?, ?it/s]
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
  0%|          | 0/1900 [00:00<?, ?it/s]

IndexError: index out of range in self

In [10]:
model_path = './model-20k'

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelWithLMHead.from_pretrained(model_path)

text_generator = pipeline(
    'text-generation', 
    model=model, 
    tokenizer=tokenizer,
    do_sample = True
    )

In [13]:
gen_text = text_generator("Have you ever", no_repeat_ngram_size=3, repetition_penalty=2.2)
print(gen_text[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Have you ever felt like a kid in the beginning – even with your father-in law?
A. It doesn't have to be true; it just needs its own little "chic." Just let's call him what he is, no


Prompt: "Have you ever"
no_repeat_ngram_size = 3
repetition_penalty = 2.2

xk meaning .  x/2 test and x/2 training   
block size = 256

- 2k prompt : `Have you ever asked your kids to read a book and write about it? We'd give them that. What's the most difficult thing someone might do instead of doing what they're reading is just writing themselves out as somebody who would only like whatever he`
- 10k prompt : `Have you ever been a parent who hasn't seen a therapist, yet you wonder a bit if you'll have to take up such a huge role in the life of a child? If so, the reason is that therapists, rather than psychiatrists or psychologists`
- 20k prompt : `Have you ever been a parent who hasn't seen this yet, and maybe your child isn't happy because of how he feels? Maybe they've thought things all along about having their best intentions.
             Here's the thing though: There`

- 100k prompt : `Have you ever been a parent who hasn't seen this yet, and maybe your child isn't happy because of how he feels? Maybe they've thought things all along about having their best intentions.
             Here's the thing though: There`